# enriched (main) Persons with Places
* write Persons (main) to .txt file
* use this file to annotate LOC within prodigy
* train a LOC-model
* run it and annotate Persons

In [ ]:
import ast
import random
from spacytei.train import batch_train

In [ ]:
persons = Person.objects.filter(is_main_person__isnull=False)
persons.count()

In [ ]:
items = [x.written_name for x in persons]
items = sorted(iter(items), key=lambda k: random.random())
filename = "person__main.txt"
with open(filename, 'w', encoding="utf-8") as f:
    for x in items:
        f.write("{}".format(x) + '\n')

In [ ]:
file = r"C:\Users\pandorfer\Documents\Redmine\prodigy\invs\invs_loc.jsonl"

In [ ]:
with open(file) as f:
    TRAIN_DATA = f.readlines()
train_data = [ast.literal_eval(x) for x in TRAIN_DATA]
print(len(train_data))
train_data[4]

In [ ]:
batch_train(train_data=train_data, output_dir='./data/main_loc', new_label='LOC', eval_split=0.2, n_iter=8)

In [ ]:
import spacy

In [ ]:
nlp = spacy.load(r"./data/main_loc")

In [ ]:
for x in persons:
    doc = nlp("{}".format(x.written_name))
    for ent in doc.ents:
        if ent.label_ == "LOC":
            place, _ = Place.objects.get_or_create(
                name="{}".format(ent.text)
            )
            x.belongs_to_place.add(place)
            break
    x.save()